In [4]:
# Limpieza de Absenteeism-data.csv según enunciado
# Pasos:
# 1) Drop 'ID'
# 2) Dummies de 'Reason for Absence' (1..28) y agrupar en 4 grupos
# 3) Eliminar 'Reason for Absence'
# 4) Extraer Month y Day of the Week desde 'Date' y eliminar 'Date'
# 5) Binarizar 'Education' (ver lógica debajo)
# 6) Guardar checkpoints y archivo final

import pandas as pd
import numpy as np
from pathlib import Path

# --- Config ---
# Ajusta estas rutas si lo necesitas:
IN_PATH  = Path("Absenteeism-data.csv")
CP1_PATH = Path("checkpoint1_after_drop_id.csv")
CP2_PATH = Path("checkpoint2_with_reason_dummies.csv")
CP3_PATH = Path("checkpoint3_grouped_reasons.csv")
CP4_PATH = Path("checkpoint4_after_date_parts.csv")
OUT_PATH = Path("df-cleaned-generated.csv")

# --- Carga ---
df = pd.read_csv(IN_PATH)

# --- 1) Eliminar 'ID' si existe ---
id_cols = [c for c in df.columns if c.strip().lower() == "id"]
if id_cols:
    df = df.drop(columns=id_cols)
df.to_csv(CP1_PATH, index=False)

# --- 2) Dummies para 'Reason for Absence' y agrupar ---
# Ubicar columna de Reason
reason_col = None
for c in df.columns:
    if "reason" in c.lower():
        reason_col = c
        break
if reason_col is None:
    raise ValueError("No se encontró la columna 'Reason for Absence'.")

# Asegurar categorías 1..28 aunque falten códigos en los datos
categories = list(range(1, 29))
reason_cat = pd.Categorical(df[reason_col], categories=categories, ordered=False)
reason_dummies = pd.get_dummies(reason_cat, prefix="Reason", prefix_sep="_", dummy_na=False)

# Guardar checkpoint con dummies anexados (auditoría)
df_with_dummies = pd.concat([df, reason_dummies], axis=1)
df_with_dummies.to_csv(CP2_PATH, index=False)

def cols_for_range(start, end):
    return [f"Reason_{i}" for i in range(start, end + 1)]

# max(axis=1) para indicar pertenencia a cualquiera de las razones de ese rango
grp1 = reason_dummies[cols_for_range(1, 14)].max(axis=1).astype(int)
grp2 = reason_dummies[cols_for_range(15, 17)].max(axis=1).astype(int)
grp3 = reason_dummies[cols_for_range(18, 21)].max(axis=1).astype(int)
grp4 = reason_dummies[cols_for_range(22, 28)].max(axis=1).astype(int)

df = df.assign(
    Reason_1=grp1,
    Reason_2=grp2,
    Reason_3=grp3,
    Reason_4=grp4
)

# --- 3) Eliminar la columna original 'Reason for Absence' ---
df = df.drop(columns=[reason_col])
df.to_csv(CP3_PATH, index=False)

# --- 4) Extraer Month y Day of the Week desde 'Date', luego eliminar 'Date' ---
# Nota: weekday: lunes=0, ..., domingo=6 (convención de pandas)
date_col = None
for c in df.columns:
    if c.strip().lower() == "date" or ("date" in c.lower()):
        date_col = c
        break
if date_col is None:
    raise ValueError("No se encontró la columna 'Date'.")

dt = pd.to_datetime(df[date_col], errors="coerce", dayfirst=True)
df["Month"] = dt.dt.month.astype("Int64")
df["Day of the Week"] = dt.dt.weekday.astype("Int64")
df = df.drop(columns=[date_col])
df.to_csv(CP4_PATH, index=False)

# --- 5) Binarizar 'Education' ---
# Enunciado: "mapear el valor 0 a 1, y el valor 1 y el resto de valores a 0".
# Sin embargo, en el dataset público típico Education es {1,2,3,4} (1=básico, 2/3/4=superior),
# donde suele mapearse 1->0 y 2/3/4->1. Para cumplir el enunciado literalmente SI YA ESTÁ EN {0,1}:
#   0 -> 1; 1 -> 0
# Si es {1,2,3,4}, aplicamos la convención habitual (1->0; 2/3/4->1).

edu_col = None
for c in df.columns:
    if "education" in c.lower():
        edu_col = c
        break
if edu_col is None:
    raise ValueError("No se encontró la columna 'Education'.")

edu_unique = pd.Series(df[edu_col].unique()).dropna().astype(float).astype(int).tolist()

def map_education(val):
    if pd.isna(val):
        return np.nan
    x = int(val)
    if set(edu_unique).issubset({0, 1}):
        # Caso binario ya existente: aplicar literal del enunciado
        return 1 if x == 0 else 0
    else:
        # Caso típico 1..4: 1->0 (básico), 2/3/4->1 (superior)
        return 0 if x == 1 else 1

df[edu_col] = df[edu_col].map(map_education).astype("Int64")

# --- 6) Guardar DataFrame final ---
df.to_csv(OUT_PATH, index=False)

print("✔ Limpieza completada.")
print(f"- Guardado: {OUT_PATH}")
print(f"- Checkpoints: {CP1_PATH}, {CP2_PATH}, {CP3_PATH}, {CP4_PATH}")

# (Opcional) Comparar contra un df-cleaned.csv de referencia si lo tienes en el mismo directorio:
REF_PATH = Path("df-cleaned.csv")
if REF_PATH.exists():
    ref = pd.read_csv(REF_PATH)
    same_shape = df.shape == ref.shape
    same_cols  = list(df.columns) == list(ref.columns)
    same_vals  = same_shape and same_cols and df.equals(ref)
    print("\nComparación con df-cleaned.csv:")
    print(f"  - Misma forma: {same_shape}")
    print(f"  - Mismo orden de columnas: {same_cols}")
    print(f"  - Datos idénticos (pandas.equals): {same_vals}")
    if not same_vals:
        # Pequeña ayuda para diagnosticar
        if not same_shape:
            print(f"    Formas -> actual: {df.shape}, ref: {ref.shape}")
        if not same_cols:
            diff_left  = [c for c in df.columns if c not in ref.columns]
            diff_right = [c for c in ref.columns if c not in df.columns]
            print(f"    En actual y no en ref: {diff_left}")
            print(f"    En ref y no en actual: {diff_right}")


FileNotFoundError: [Errno 2] No such file or directory: 'Absenteeism-data.csv'